In [5]:
import pyomo.environ as pyo

In [6]:
model = pyo.ConcreteModel()

# 변수 정의
model.x1 = pyo.Var(domain=pyo.PositiveReals)
model.x2 = pyo.Var(domain=pyo.PositiveReals)

# 목적함수 정의
model.obj = pyo.Objective(expr=3*model.x1+2*model.x2, sense=pyo.maximize)

# 제약조건 정의
model.con1 = pyo.Constraint(expr=model.x1+model.x2 <= 80)
model.con2 = pyo.Constraint(expr=2*model.x1+model.x2 <= 100)
model.con3 = pyo.Constraint(expr=model.x1 <= 40)

# ipopt 솔버로 모델 계산
solver = pyo.SolverFactory('ipopt')
result = solver.solve(model, tee=False)

# 결과출력
print("x1 =", round(pyo.value(model.x1), 2))
print("x2 =", round(pyo.value(model.x2), 2))
print("con1 =", round(pyo.value(model.con1.body), 2), "<=", model.con1.upper)
print("con2 =", round(pyo.value(model.con2.body), 2), "<=", model.con2.upper)
print("con3 =", round(pyo.value(model.con3.body), 2), "<=", model.con3.upper)
print("목적함수 값 =", round(pyo.value(model.obj), 2))

x1 = 20.0
x2 = 60.0
con1 = 80.0 <= 80.0
con2 = 100.0 <= 100.0
con3 = 20.0 <= 40.0
목적함수 값 = 180.0


In [7]:
import pyomo.environ as pyo

# 변수와 계수 정의
var_names = ['x1', 'x2']
obj_coeff = {'x1': 3, 'x2': 2}
con_coeff = {
    'con1': {'x1': 1, 'x2': 1},
    'con2': {'x1': 2, 'x2': 1},
    'con3': {'x1': 1, 'x2': 0}
}
con_rhs = {'con1': 80, 'con2': 100, 'con3': 40}

# 모델 생성
model2 = pyo.ConcreteModel()

# 변수 집합
model2.VARS = pyo.Set(initialize=var_names)

# 변수 정의
model2.x = pyo.Var(model2.VARS, domain=pyo.PositiveReals)

# 목적함수
def obj_rule(m):
    return sum(obj_coeff[v] * m.x[v] for v in m.VARS)
model2.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# 제약조건
def con1_rule(m):
    return sum(con_coeff['con1'][v] * m.x[v] for v in m.VARS) <= con_rhs['con1']
model2.con1 = pyo.Constraint(rule=con1_rule)

def con2_rule(m):
    return sum(con_coeff['con2'][v] * m.x[v] for v in m.VARS) <= con_rhs['con2']
model2.con2 = pyo.Constraint(rule=con2_rule)

def con3_rule(m):
    return sum(con_coeff['con3'][v] * m.x[v] for v in m.VARS) <= con_rhs['con3']
model2.con3 = pyo.Constraint(rule=con3_rule)

# ipopt 솔버로 모델 계산
solver2 = pyo.SolverFactory('ipopt')
result2 = solver2.solve(model2, tee=False)

# 결과 출력
for v in model2.VARS:
    print(f"{v} =", round(pyo.value(model2.x[v]), 2))
print("con1 =", round(sum(con_coeff['con1'][v]*pyo.value(model2.x[v]) for v in model2.VARS),2), "<=", con_rhs['con1'])
print("con2 =", round(sum(con_coeff['con2'][v]*pyo.value(model2.x[v]) for v in model2.VARS),2), "<=", con_rhs['con2'])
print("con3 =", round(sum(con_coeff['con3'][v]*pyo.value(model2.x[v]) for v in model2.VARS),2), "<=", con_rhs['con3'])
print("목적함수 값 =", round(pyo.value(model2.obj), 2))


x1 = 20.0
x2 = 60.0
con1 = 80.0 <= 80
con2 = 100.0 <= 100
con3 = 20.0 <= 40
목적함수 값 = 180.0
